In [40]:
###################
#### Prompts   ####
###################

prompts = {}
prompts['identifier'] = 'Fasse den Text zusammen.'
# ...

In [41]:
import io
###################
#### Functions ####
###################

from openai import OpenAI
import os
import json

from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
gpt_model = os.getenv('GPT_MODEL')


client = OpenAI(
    api_key=openai_api_key,
)


def build_message(prompt, input_text):
    messages = [
        {"role": "user", "content": prompt},
        {"role": "user", "content": f'"{input_text}"'},
    ]
    return messages

def run_completion(prompts, input_text):
    result = {}
    for key, value in prompts.items():
        print(f'Run completion "{key}"')
        try:
            completion_result = client.chat.completions.create(
                model=os.getenv('GPT_MODEL'),
                messages=build_message(value, input_text)
            )
            assistant = completion_result.choices[0].message.content if completion_result else ''
            assistant_result = assistant.strip('"').strip('\'').strip()
            result[key] = assistant_result
        except Exception as e:
            error_message = f'Error occurred while complete "{key}": {str(e)}'
            print(error_message)
    return result

def write_to_output(filename, data):
    current_folder = os.getcwd()
    output_folder_path = os.path.join(current_folder, 'output')
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    file_path = os.path.join(output_folder_path, f'{filename}.json')
    if os.path.exists(file_path):
        os.remove(file_path)

    print(f'Write to file "{filename}"')
    with io.open(file_path, 'w', encoding='utf-8') as outfile:
        json.dump(data, outfile, ensure_ascii=False)

def read_input():
    current_folder = os.getcwd()
    input_folder = os.path.join(current_folder, 'input')
    input_data = {}

    if os.path.exists(input_folder) and os.path.isdir(input_folder):
        for filename in os.listdir(input_folder):
            if filename.endswith('.txt'):
                print(f'Read file "{filename}"')
                file_path = os.path.join(input_folder, filename)
                with io.open(file_path, 'r', encoding='utf-8') as file:
                    file_content = file.read()
                key = os.path.splitext(filename)[0]
                input_data[key] = file_content

    return input_data


In [42]:
###################
#### Execution ####
###################

input_data = read_input()
for filename, text in input_data.items():
    print(f'Processing file "{filename}"')
    completion_result = run_completion(prompts, text)
    write_to_output(filename, completion_result)
print('Finished')


Read file "example.txt"
Processing file "example"
Run completion "identifier"
Write to file "example"
Finished
